In [1]:
from tz.osemosys import Model

model = Model.from_yaml("../models/PAK/")

model.solve()

ValueError: invalid literal for int() with base 10: '*'

In [ ]:
# import pandas as pd

# dfs = []
# for i in list( model._data['TECHNOLOGY'].values ):
#     dfs.append(model.solution.sel(TECHNOLOGY=i).Production.to_dataframe())

# ( pd
#  .concat(dfs, axis=0, ignore_index=False)
#  .reset_index()
#  .pivot_table(index='YEAR',columns='TECHNOLOGY',values='Production',aggfunc='sum')
# )

In [ ]:
import pandas as pd

dfs = []
for i in list( model._data['TECHNOLOGY'].values ):
    dfs.append(model._m.solution.sel(TECHNOLOGY=i).RateOfActivity.to_dataframe())

( pd
 .concat(dfs, axis=0, ignore_index=False)
 .reset_index()
 .pivot_table(index='YEAR',columns='TECHNOLOGY',values='RateOfActivity',aggfunc='sum')
 .mul(1/6)
 .div(7)
)

In [ ]:
model._m.solution.NewStorageCapacity.to_dataframe()

In [ ]:
( model
 ._m
 .solution
 .RateOfActivity
 .to_dataframe()
 .reset_index()
 .query(" MODE_OF_OPERATION.isin(['CHARGE', 'DISCHARGE'])")
 .query(" TECHNOLOGY == 'battery' ")
 .groupby(by='MODE_OF_OPERATION')
 .sum(numeric_only=True)
)

In [ ]:
model._m.solution.TradeAnnual.to_dataframe().reset_index().query("FUEL == 'secondary-electricity' ").TradeAnnual.describe()

In [ ]:
model._m.solution

In [ ]:
from tz.osemosys import Model

# run model
iso_code = 'PAK'
model = Model.from_yaml(f"../models/{iso_code}/")
model.solve()

In [ ]:
model.solution.RateOfActivity.to_dataframe()

In [ ]:
for i in list( model.solution.variables ):
    if 'Emission' in i:
        print(i)

In [ ]:
# plot generation
gen = (
    model
    .solution
    .Production
    .to_dataframe()
    .reset_index()
    .pivot_table(
        index='YEAR', 
        columns='REGION', 
        values='Production',
        aggfunc='sum',
    )
)

gen.plot()